In [0]:
%pip install azure-storage-blob


Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-994cc749-c4f9-4331-b638-badef3e7ad8b
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
%run "/Users/anaparrillo03@gmail.com/Carga de Datos"

In [0]:
df_E = df_Usa.toPandas()
df_M = df_Mexico.toPandas()
df_J = df_Japon.toPandas()
df_P = df_Peru.toPandas()

In [0]:
# Importar funciones y librerias
from pyspark.sql.functions import *
import seaborn as sns
import pandas as pd
import matplotlib as plt
import warnings
warnings.filterwarnings("ignore")

## Tranformaciones base Perú

In [0]:
# Cambiar tipo de Dato
df_P['Magnitud'] = pd.to_numeric(df_P['Magnitud'])
df_P["Profundidad"] = pd.to_numeric(df_P["Profundidad"])
df_P['Fecha'] = pd.to_datetime(df_P['Fecha'], errors='ignore')

In [0]:
df_P['Longitud'] = df_P['Longitud'].astype(str)
df_P['Latitud'] = df_P['Latitud'].astype(str)
df_P['Coordenadas'] = df_P['Latitud'] + ', ' + df_P['Longitud']
df_P['País'] = 'Perú'

In [0]:
!pip install reverse_geocoder
import reverse_geocoder as rg

In [0]:
df_P

,Fecha,Hora,Latitud,Longitud,Profundidad,Magnitud,Coordenadas,País
0,2008-05-20,21:37:11.00,-8.6391,-74.7701,135.0,5.7,"-8.6391, -74.7701",Perú
1,2008-07-06,13:06:29.00,-12.4652,-77.3985,66.0,5.3,"-12.4652, -77.3985",Perú
2,2008-01-26,12:03:18.00,-5.947,-75.4306,156.0,4.5,"-5.947, -75.4306",Perú
3,2008-10-01,21:46:29.00,-7.1948,-75.1474,170.0,4.5,"-7.1948, -75.1474",Perú
4,2008-08-03,23:51:10.00,-13.348,-76.8272,29.0,5.2,"-13.348, -76.8272",Perú
...,...,...,...,...,...,...,...,...
10884,2023-03-27,20:13:13,-10.53,-74.7,134.0,4.6,"-10.53, -74.7",Perú
10885,2022-10-28,23:19:58,-3.76,-76.53,110.0,4.8,"-3.76, -76.53",Perú
10886,2023-05-22,00:05:45,-15.81,-73.32,92.0,4.2,"-15.81, -73.32",Perú
10887,2022-10-28,23:35:13,-14.61,-76.41,15.0,5.2,"-14.61, -76.41",Perú


In [0]:
# Función Dirección Departamento
def get_location_D(row):
   lat, lon = row.split(',')
   Departamento = rg.search([(float(lat), float(lon))])
   return Departamento[0]['admin1']

In [0]:
df_P['Locación'] = df_P['Coordenadas'].apply(get_location_D)

Loading formatted geocoded file...


In [0]:
df_P = df_P.drop('Coordenadas', axis=1)

In [0]:
rangos = [(2, 2.9), (3, 3.9), (4, 4.9), (5, 5.9), (6, 6.9), (7, 7.9), (8, 9.9), (10, 15)]
etiquetas = ['Micro Magnitud', 'Menor Magnitud', 'Ligera Magnitud', 'Moderada Magnitud',
              'Fuerte Magnitud', 'Mayor Magnitud', 'Gran Magnitud', 'Magnitud Épica']

df_P['Clasificación Magnitud'] = pd.cut(df_P['Magnitud'], bins=[lim[0] for lim in rangos] + [rangos[-1][1]],
                               labels=etiquetas, right=False)

In [0]:
rangos = [(0, 70), (70, 300), (300, 1500)]
etiquetas = ['Superficial', 'Media', 'Profundo']

df_P['Clasificación Profundidad'] = pd.cut(df_P['Profundidad'], bins=[lim[0] for lim in rangos] + [rangos[-1][1]],
                               labels=etiquetas, right=False)

In [0]:
df_P = df_P.reindex(columns=['Fecha','Hora','Latitud','Longitud','País','Locación','Profundidad',
                             'Clasificación Profundidad','Magnitud','Clasificación Magnitud'])

In [0]:
df_P.rename(columns={'Locación':'Locacion'}, inplace = True)

In [0]:
df_P['Fecha'] = pd.to_datetime(df_P['Fecha'])

In [0]:
df_P['Hora']

Out[22]: 0        21:37:11.00
1        13:06:29.00
2        12:03:18.00
3        21:46:29.00
4        23:51:10.00
            ...     
10884       20:13:13
10885       23:19:58
10886       00:05:45
10887       23:35:13
10888       12:32:40
Name: Hora, Length: 10889, dtype: object

In [0]:
Base_Final_Peru = df_P

In [0]:
Base_Final_Peru

,Fecha,Hora,Latitud,Longitud,País,Locacion,Profundidad,Clasificación Profundidad,Magnitud,Clasificación Magnitud
0,2008-05-20,21:37:11.00,-8.6391,-74.7701,Perú,Huanuco,135.0,Media,5.7,Moderada Magnitud
1,2008-07-06,13:06:29.00,-12.4652,-77.3985,Perú,Lima,66.0,Superficial,5.3,Moderada Magnitud
2,2008-01-26,12:03:18.00,-5.947,-75.4306,Perú,Loreto,156.0,Media,4.5,Ligera Magnitud
3,2008-10-01,21:46:29.00,-7.1948,-75.1474,Perú,San Martin,170.0,Media,4.5,Ligera Magnitud
4,2008-08-03,23:51:10.00,-13.348,-76.8272,Perú,Lima,29.0,Superficial,5.2,Moderada Magnitud
...,...,...,...,...,...,...,...,...,...,...
10884,2023-03-27,20:13:13,-10.53,-74.7,Perú,Junin,134.0,Media,4.6,Ligera Magnitud
10885,2022-10-28,23:19:58,-3.76,-76.53,Perú,Loreto,110.0,Media,4.8,Ligera Magnitud
10886,2023-05-22,00:05:45,-15.81,-73.32,Perú,Arequipa,92.0,Media,4.2,Ligera Magnitud
10887,2022-10-28,23:35:13,-14.61,-76.41,Perú,Ica,15.0,Superficial,5.2,Moderada Magnitud


## Transformación Base EEUU

In [0]:
# Definir la función para determinar la peligrosidad
def determinar_peligrosidad(magnitud):
    if magnitud < 2.0:
        return 'Micro Magnitud'
    elif magnitud < 3.0:
        return 'Menor Magnitud'
    elif magnitud < 4.0:
        return 'Ligera Magnitud'
    elif magnitud < 5.0:
        return 'Moderada Magnitud'
    elif magnitud < 6.0:
        return 'Fuerte Magnitud'
    elif magnitud < 7.0:
        return 'Mayor Magnitud'
    elif magnitud < 8.0:
        return 'Gran Magnitud'
    else:
        return 'Magnitud Épica'

    # Convertir la columna "Magnitud" a tipo float si es necesario

df_E['Magnitud'] = df_E['Magnitud'].astype(float)

# Aplicar la función a cada fila de la columna "Magnitud" y crear la nueva columna "peligrosidad"
df_E['Clasificación Magnitud'] = df_E['Magnitud'].apply(determinar_peligrosidad)

In [0]:
def clasificar_profundidad(profundidad):

    if profundidad <= 10:
        return "Superficial"
    elif profundidad > 10 and profundidad <= 30:
        return"Media"
    elif profundidad > 30:
        return"Profundo"

df_E['Profundidad'] = df_E['Profundidad'].astype(float)  # Convertir a tipo float

# Aplicar la función a cada fila de la columna "Profundidad" y crear la nueva columna "Clasificación Profundidad"
df_E['Clasificación Profundidad'] = df_E['Profundidad'].apply(clasificar_profundidad)

In [0]:
# Lista de columnas a eliminar
columnas_eliminar = ["Tipo", "Estatus", "Tsunami","Fuente","Significancia","Ubicacion"]

# Eliminar las columnas en Databricks
df_E = df_E.drop(columnas_eliminar, axis = 1)

In [0]:
df_E['Fecha_1'] = pd.to_datetime(df_E['Fecha'], unit='ms').dt.date
df_E['Hora'] = pd.to_datetime(df_E['Fecha'], unit='ms').dt.strftime('%H:%M:%S')
df_E = df_E.drop(columns='Fecha', axis = 1)
df_E.rename(columns={'Fecha_1':'Fecha'}, inplace= True)
df_E['País'] = 'EEUU'

In [0]:
def Cambio_estado(obj):
    if obj == 1:
        obj = 'California'
        return obj
    elif obj == 2:
        obj = 'Washington'
        return obj
    elif obj == 3:
        obj = 'Oregon'
        return obj
    else:
        obj = 'Alaska'  
        return obj
        

In [0]:
df_E

,Id,Magnitud,Profundidad,Ubicacion,Latitud,Longitud,Tipo,Estatus,Tsunami,Significancia,Fuente,Clasificación Magnitud,Clasificación Profundidad,Fecha,Hora,País
0,1,2.51,7.82,"12km SW of Searles Valley, CA",35.6848333,-117.483,earthquake,reviewed,0,97,ci,Menor Magnitud,Superficial,2023-06-21,13:58:59,EEUU
1,1,5.80,4.66,"18km SSE of Lone Pine, CA",36.4468333,-117.9751667,earthquake,reviewed,1,1098,ci,Fuerte Magnitud,Superficial,2020-06-24,17:40:49,EEUU
2,1,2.77,3.69,"20km E of Little Lake, CA",35.9531667,-117.6913333,earthquake,reviewed,0,118,ci,Menor Magnitud,Superficial,2023-05-21,10:09:07,EEUU
3,1,2.82,20.63,"6km SW of Antioch, CA",37.9575,-121.846,earthquake,reviewed,0,133,nc,Menor Magnitud,Media,2023-04-23,20:20:18,EEUU
4,1,2.86,4.10,"11km S of Tres Pinos, CA",36.688,-121.3255,earthquake,reviewed,0,128,nc,Menor Magnitud,Superficial,2023-04-18,17:23:53,EEUU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13011,4,2.50,76.20,"15 km NNE of Skwentna, Alaska",62.1141,-151.2852,earthquake,reviewed,0,96,ak,Menor Magnitud,Profundo,2008-02-16,00:57:29,EEUU
13012,4,2.50,7.40,"53 km SSE of Denali National Park, Alaska",63.0842,-151.3811,earthquake,reviewed,0,96,ak,Menor Magnitud,Superficial,2008-02-06,12:11:28,EEUU
13013,4,2.50,11.30,"41 km SE of Denali National Park, Alaska",63.2929,-151.1068,earthquake,reviewed,0,96,ak,Menor Magnitud,Media,2008-01-31,05:06:05,EEUU
13014,4,3.00,128.20,"60 km NNW of Petersville, Alaska",63.0307,-151.0047,earthquake,reviewed,0,138,ak,Ligera Magnitud,Profundo,2008-01-19,21:27:10,EEUU


In [0]:
df_E['Id'] = df_E['Id'].apply(Cambio_estado)
df_E.rename(columns={'Id':'Locación'}, inplace= True) 
df_E = df_E.reindex(columns = ['Fecha','Hora','Latitud','Longitud','País','Locación','Profundidad',
                         'Clasificación Profundidad', 'Magnitud', 'Clasificación Magnitud'])

df_E = df_E.sort_values('Fecha')
df_E = df_E.reset_index(drop=True)


In [0]:
df_E.rename(columns={'Locación':'Locacion'}, inplace=True)

In [0]:
df_E['Fecha'] = pd.to_datetime(df_E['Fecha'])

In [0]:
Base_Final_EEUU = df_E

In [0]:
Base_Final_EEUU

,Fecha,Hora,Latitud,Longitud,País,Locacion,Profundidad,Clasificación Profundidad,Magnitud,Clasificación Magnitud
0,2008-01-01,00:34:53,63.5818,-150.0199,EEUU,Alaska,132.700,Profundo,2.60,Menor Magnitud
1,2008-01-03,04:21:53,45.127,-120.946,EEUU,Oregon,16.162,Media,2.70,Menor Magnitud
2,2008-01-03,04:21:53,45.127,-120.946,EEUU,Washington,16.162,Media,2.70,Menor Magnitud
3,2008-01-04,04:38:59,63.652,-149.5325,EEUU,Alaska,117.300,Profundo,3.10,Ligera Magnitud
4,2008-01-07,22:02:08,48.0623333,-120.8096667,EEUU,Washington,-0.300,Superficial,2.70,Menor Magnitud
...,...,...,...,...,...,...,...,...,...,...
13011,2023-06-21,14:26:08,62.4333,-150.5708,EEUU,Alaska,63.700,Profundo,3.80,Ligera Magnitud
13012,2023-06-21,21:01:18,45.6715,-122.71283333333334,EEUU,Washington,20.070,Media,2.54,Menor Magnitud
13013,2023-06-21,13:58:59,35.6848333,-117.483,EEUU,California,7.820,Superficial,2.51,Menor Magnitud
13014,2023-06-22,08:04:45,46.629333333333335,-121.95766666666668,EEUU,Washington,8.170,Superficial,2.64,Menor Magnitud


##Transformación Base México

In [0]:
df_M

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Locacion,Fecha_UTC,Hora_UTC,Estatus
0,2008-01-04,1970-01-01 16:25:43,4.1,19.2,-104.75,46.0,"20 km al OESTE de CIHUATLAN, JAL",2008-01-04,1970-01-01 22:25:43,revisado
1,2009-08-03,1970-01-01 13:40:54,5.8,29.42,-113.5,10.0,"171 km al NORESTE de GUERRERO NEGRO, BCS",2009-08-03,1970-01-01 18:40:54,revisado
2,2008-01-06,1970-01-01 03:10:22,3.8,16.87,-94.91,20.0,"14 km al ESTE de MATIAS ROMERO, OAX",2008-01-06,1970-01-01 09:10:22,revisado
3,2008-01-01,1970-01-01 14:07:14,3.4,16.82,-100.19,5.0,"23 km al SUROESTE de COYUCA DE BENITEZ, GRO",2008-01-01,1970-01-01 20:07:14,revisado
4,2009-05-01,1970-01-01 18:38:09,2.7,17.01,-100.08,7.0,"1 km al NORESTE de COYUCA DE BENITEZ, GRO",2009-05-01,1970-01-01 23:38:09,revisado
...,...,...,...,...,...,...,...,...,...,...
9995,2011-07-08,1970-01-01 00:15:23,3.6,31.15,-115.36,38.0,"52 km al NOROESTE de SAN FELIPE, BC",2011-07-08,1970-01-01 05:15:23,revisado
9996,2011-07-08,1970-01-01 14:05:24,3.6,14.2,-93.08,20.0,"113 km al SUROESTE de CD HIDALGO, CHIS",2011-07-08,1970-01-01 19:05:24,revisado
9997,2011-07-08,1970-01-01 15:51:18,3.6,15.35,-93.1,89.0,"25 km al SUROESTE de MAPASTEPEC, CHIS",2011-07-08,1970-01-01 20:51:18,revisado
9998,2011-07-09,1970-01-01 04:13:36,4.3,14.32,-92.93,19.0,"93 km al SUROESTE de CD HIDALGO, CHIS",2011-07-09,1970-01-01 09:13:36,revisado


In [0]:
df_M.drop(["Estatus"],axis=1,inplace=True)
df_M["País"] = 'México'

In [0]:
df_M.dropna(inplace=True)
df_M.reset_index(inplace=True,drop=True)

In [0]:
for indice in (range(len(df_M))):
    estado = df_M["Locacion"][indice].split(",")[-1].strip()

    if estado == "OAX":
        df_M.loc[indice,"Locacion"] = "Oaxaca"
    elif estado == "GRO":
        df_M.loc[indice,"Locacion"] = "Guerrero"
    elif estado == "CHIS":
        df_M.loc[indice,"Locacion"] = "Chiapas"
    elif estado == "MICH":
        df_M.loc[indice,"Locacion"] = "Michoacán"
    elif estado == "BC":
        df_M.loc[indice,"Locacion"] = "Baja California"
    elif estado == "JAL":
        df_M.loc[indice,"Locacion"] = "Jalisco"
    elif estado == "COL":
        df_M.loc[indice,"Locacion"] = "Colima" 
    elif estado == "VER":
        df_M.loc[indice,"Locacion"] = "Veracruz"
    elif estado == "BCS":
        df_M.loc[indice,"Locacion"] = "Baja California Sur"
    elif estado == "SON":
        df_M.loc[indice,"Locacion"] = "Sonora"
    elif estado == "PUE":
        df_M.loc[indice,"Locacion"] = "Puebla"
    elif estado == "SIN":
        df_M.loc[indice,"Locacion"] = "Sinaloa"
    elif estado == "HGO":
        df_M.loc[indice,"Locacion"] = "Hidalgo"
    
    elif estado == "CHIH":
        df_M.loc[indice,"Locacion"] = "Chihuahua"
    elif estado == "MEX":
        df_M.loc[indice,"Locacion"] = "Mexico"
    elif estado == "NL":
        df_M.loc[indice,"Locacion"] = "Nuevo León"
    elif estado == "ZAC":
        df_M.loc[indice,"Locacion"] = "Zacatecas"
    elif estado == "TAB":
        df_M.loc[indice,"Locacion"] = "Tabasco"
    elif estado == "NAY":
        df_M.loc[indice,"Locacion"] = "Nayarit"
    elif estado == "MOR":
        df_M.loc[indice,"Locacion"] = "Morelos"
    elif estado == "CDMX":
        df_M.loc[indice,"Locacion"] = "Ciudad de México"
    elif estado == "SLP":
        df_M.loc[indice,"Locacion"] = "San Luis Potosí"
    elif estado == "COAH":
        df_M.loc[indice,"Locacion"] = "Coahuila de Zaragoza"
    elif estado == "TAMS":
        df_M.loc[indice,"Locacion"] = "Tamaulipas"
    elif estado == "GTO":
        df_M.loc[indice,"Locacion"] = "Guanajuato"

    elif estado == "TLAX":
        df_M.loc[indice,"Locacion"] = "Tlaxcala"
    elif estado == "DGO":
        df_M.loc[indice,"Locacion"] = "Durango"
    elif estado == "AGS":
        df_M.loc[indice,"Locacion"] = "Aguascalientes"
    elif estado == "QRO":
        df_M.loc[indice,"Locacion"] = "Querétaro"
    elif estado == "QR":
        df_M.loc[indice,"Locacion"] = "Quintana Roo"
    elif estado == "CAMP":
        df_M.loc[indice,"Locacion"] = "Campeche"
    elif estado == "YUC":
        df_M.loc[indice,"Locacion"] = "Yucatán"
    elif estado == "N":
        df_M.loc[indice,"Locacion"] = "Nuevo León" 


In [0]:
lista = df_M[(df_M["Profundidad"]=="en revision") | (df_M["Profundidad"]=="menos de 1") | (df_M["Magnitud"]=="no calculable")].index
df_M.drop(index=lista,inplace=True)
df_M.reset_index(inplace=True,drop=True)
df_M['Profundidad'] = df_M['Profundidad'].astype(float)  # Convertir a tipo float
# Aplicar la función a cada fila de la columna "Profundidad" y crear la nueva columna "Clasificación Profundidad"
df_M['Clasificación Profundidad'] = df_M['Profundidad'].apply(clasificar_profundidad)

In [0]:
rangos = [(2, 2.9), (3, 3.9), (4, 4.9), (5, 5.9), (6, 6.9), (7, 7.9), (8, 9.9), (10, 15)]
etiquetas = ['Micro Magnitud', 'Menor Magnitud', 'Ligera Magnitud', 'Moderada Magnitud',
              'Fuerte Magnitud', 'Mayor Magnitud', 'Gran Magnitud', 'Magnitud Épica']

df_M['Clasificación Magnitud'] = pd.cut(df_M['Magnitud'], bins=[lim[0] for lim in rangos] + [rangos[-1][1]],
                               labels=etiquetas, right=False)

In [0]:
df_M

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Locacion,Fecha_UTC,Hora_UTC,País,Clasificación Profundidad,Clasificación Magnitud
0,2008-01-04,1970-01-01 16:25:43,4.1,19.2,-104.75,46.0,Jalisco,2008-01-04,1970-01-01 22:25:43,México,Profundo,Ligera Magnitud
1,2009-08-03,1970-01-01 13:40:54,5.8,29.42,-113.5,10.0,Baja California Sur,2009-08-03,1970-01-01 18:40:54,México,Superficial,Moderada Magnitud
2,2008-01-06,1970-01-01 03:10:22,3.8,16.87,-94.91,20.0,Oaxaca,2008-01-06,1970-01-01 09:10:22,México,Media,Menor Magnitud
3,2008-01-01,1970-01-01 14:07:14,3.4,16.82,-100.19,5.0,Guerrero,2008-01-01,1970-01-01 20:07:14,México,Superficial,Menor Magnitud
4,2009-05-01,1970-01-01 18:38:09,2.7,17.01,-100.08,7.0,Guerrero,2009-05-01,1970-01-01 23:38:09,México,Superficial,Micro Magnitud
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2011-07-08,1970-01-01 00:15:23,3.6,31.15,-115.36,38.0,Baja California,2011-07-08,1970-01-01 05:15:23,México,Profundo,Menor Magnitud
9996,2011-07-08,1970-01-01 14:05:24,3.6,14.2,-93.08,20.0,Chiapas,2011-07-08,1970-01-01 19:05:24,México,Media,Menor Magnitud
9997,2011-07-08,1970-01-01 15:51:18,3.6,15.35,-93.1,89.0,Chiapas,2011-07-08,1970-01-01 20:51:18,México,Profundo,Menor Magnitud
9998,2011-07-09,1970-01-01 04:13:36,4.3,14.32,-92.93,19.0,Chiapas,2011-07-09,1970-01-01 09:13:36,México,Media,Ligera Magnitud


In [0]:
df_M.drop(columns=['Fecha_UTC','Hora_UTC'], inplace = True)
df_M['Fecha'] = pd.to_datetime(df_M['Fecha'])

In [0]:
df_M = df_M.reindex(columns = ['Fecha','Hora','Latitud','Longitud','País','Locacion','Profundidad',
                         'Clasificación Profundidad', 'Magnitud', 'Clasificación Magnitud'])

In [0]:
df_M

,Fecha,Hora,Latitud,Longitud,País,Locacion,Profundidad,Clasificación Profundidad,Magnitud,Clasificación Magnitud
0,2008-01-04,1970-01-01 16:25:43,19.2,-104.75,México,Jalisco,46.0,Profundo,4.1,Ligera Magnitud
1,2009-08-03,1970-01-01 13:40:54,29.42,-113.5,México,Baja California Sur,10.0,Superficial,5.8,Moderada Magnitud
2,2008-01-06,1970-01-01 03:10:22,16.87,-94.91,México,Oaxaca,20.0,Media,3.8,Menor Magnitud
3,2008-01-01,1970-01-01 14:07:14,16.82,-100.19,México,Guerrero,5.0,Superficial,3.4,Menor Magnitud
4,2009-05-01,1970-01-01 18:38:09,17.01,-100.08,México,Guerrero,7.0,Superficial,2.7,Micro Magnitud
...,...,...,...,...,...,...,...,...,...,...
9995,2011-07-08,1970-01-01 00:15:23,31.15,-115.36,México,Baja California,38.0,Profundo,3.6,Menor Magnitud
9996,2011-07-08,1970-01-01 14:05:24,14.2,-93.08,México,Chiapas,20.0,Media,3.6,Menor Magnitud
9997,2011-07-08,1970-01-01 15:51:18,15.35,-93.1,México,Chiapas,89.0,Profundo,3.6,Menor Magnitud
9998,2011-07-09,1970-01-01 04:13:36,14.32,-92.93,México,Chiapas,19.0,Media,4.3,Ligera Magnitud


In [0]:
df_M['Hora'] = df_M['Hora'].dt.strftime('%H:%M:%S')


In [0]:
Base_Final_Mexico = df_M

In [0]:
Base_Final_Mexico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Fecha                      10000 non-null  datetime64[ns]
 1   Hora                       10000 non-null  object        
 2   Latitud                    10000 non-null  object        
 3   Longitud                   10000 non-null  object        
 4   País                       10000 non-null  object        
 5   Locacion                   10000 non-null  object        
 6   Profundidad                10000 non-null  float64       
 7   Clasificación Profundidad  10000 non-null  object        
 8   Magnitud                   10000 non-null  float64       
 9   Clasificación Magnitud     9995 non-null   category      
dtypes: category(1), datetime64[ns](1), float64(2), object(6)
memory usage: 713.4+ KB


##Transformación Base Japón

In [0]:
df_J

,Time,Latitude,Longitude,Depth,Mag,Nst,Gap,Place
0,2023-06-15T01:13:05.542Z,33.5871,141.1257,35,4.4,29,130,"188 km SSE of Katsuura, Japan"
1,2023-06-11T09:54:44.886Z,42.5268,141.9005,121,6.2,143,31,"20 km WSW of Biratori, Japan"
2,2023-06-17T14:10:16.472Z,38.9607,144.8172,10,4.5,19,198,"253 km ESE of Yamada, Japan"
3,2023-06-25T01:12:51.284Z,29.9147,139.9913,10,5,46,125,"Izu Islands, Japan region"
4,2023-06-18T09:46:54.634Z,34.4987,143.0695,10,4.4,36,146,"245 km SE of Hasaki, Japan"
...,...,...,...,...,...,...,...,...
15979,2008-04-10T20:32:54.700Z,38.888,142.152,48.3,4.2,48,122.4,"42 km ESE of ?funato, Japan"
15980,2008-03-16T19:20:40.540Z,43.595,133.775,431.3,4.6,10,131.2,"25 km NNW of Lazo, Russia"
15981,2008-02-21T13:05:10.190Z,44.342,141.266,247,3.7,6,127.3,"54 km NW of Rumoi, Japan"
15982,2008-02-05T02:07:07.710Z,37.103,135.025,377.4,3.2,5,120.4,"140 km NW of Mikuni, Japan"


In [0]:
df_J["Time"] = pd.to_datetime(df_J["Time"])
df_J["Fecha"] = df_J["Time"].dt.strftime("%d/%m/%Y")
df_J["Hora"] = df_J["Time"].dt.strftime("%H:%M:%S")

In [0]:
def obtener_pais(ubicacion):
    if isinstance(ubicacion, str):
        pais = ubicacion.split(',')[-1].strip()
        return pais
    else:
        return 'Desconocido'

In [0]:
df_J["País"] = df_J["Place"].apply(obtener_pais)

In [0]:
df_J

,Time,Latitude,Longitude,Depth,Mag,Nst,Gap,Place,Fecha,Hora,País
0,2023-06-15 01:13:05.542000+00:00,33.5871,141.1257,35,4.4,29,130,"188 km SSE of Katsuura, Japan",15/06/2023,01:13:05,Japan
1,2023-06-11 09:54:44.886000+00:00,42.5268,141.9005,121,6.2,143,31,"20 km WSW of Biratori, Japan",11/06/2023,09:54:44,Japan
2,2023-06-17 14:10:16.472000+00:00,38.9607,144.8172,10,4.5,19,198,"253 km ESE of Yamada, Japan",17/06/2023,14:10:16,Japan
3,2023-06-25 01:12:51.284000+00:00,29.9147,139.9913,10,5,46,125,"Izu Islands, Japan region",25/06/2023,01:12:51,Japan region
4,2023-06-18 09:46:54.634000+00:00,34.4987,143.0695,10,4.4,36,146,"245 km SE of Hasaki, Japan",18/06/2023,09:46:54,Japan
...,...,...,...,...,...,...,...,...,...,...,...
15979,2008-04-10 20:32:54.700000+00:00,38.888,142.152,48.3,4.2,48,122.4,"42 km ESE of ?funato, Japan",10/04/2008,20:32:54,Japan
15980,2008-03-16 19:20:40.540000+00:00,43.595,133.775,431.3,4.6,10,131.2,"25 km NNW of Lazo, Russia",16/03/2008,19:20:40,Russia
15981,2008-02-21 13:05:10.190000+00:00,44.342,141.266,247,3.7,6,127.3,"54 km NW of Rumoi, Japan",21/02/2008,13:05:10,Japan
15982,2008-02-05 02:07:07.710000+00:00,37.103,135.025,377.4,3.2,5,120.4,"140 km NW of Mikuni, Japan",05/02/2008,02:07:07,Japan


In [0]:
df_J["País"] = df_J["País"].replace("Japan region", "Japan")


In [0]:
df_J["Localidad"] = df_J["Place"].str.split(",").str[0].str.split("of").str[-1].str.strip()


In [0]:
columnas_eliminar = ["Nst", "Gap", "Time", "Place"]
df_J = df_J.drop(columns=columnas_eliminar, axis = 1)

In [0]:
df_J = df_J.rename(columns={"Localidad" : "Locacion"})
df_J["Latitude"] = pd.to_numeric(df_J["Latitude"])
df_J["Longitude"] = pd.to_numeric(df_J["Longitude"])
df_J["Mag"] = pd.to_numeric(df_J["Mag"])
df_J["Depth"] = pd.to_numeric(df_J["Depth"])
df_J["Fecha"] = pd.to_datetime(df_J["Fecha"])

In [0]:
df_J.rename(columns={'Latitude':'Latitud','Longitude':'Longitud','Mag':'Magnitud','Depth':'Profundidad'}, inplace= True)


In [0]:
df_J

,Latitud,Longitud,Profundidad,Magnitud,Fecha,Hora,País,Locacion
0,33.5871,141.1257,35.0,4.4,2023-06-15,01:13:05,Japan,Katsuura
1,42.5268,141.9005,121.0,6.2,2023-11-06,09:54:44,Japan,Biratori
2,38.9607,144.8172,10.0,4.5,2023-06-17,14:10:16,Japan,Yamada
3,29.9147,139.9913,10.0,5.0,2023-06-25,01:12:51,Japan,Izu Islands
4,34.4987,143.0695,10.0,4.4,2023-06-18,09:46:54,Japan,Hasaki
...,...,...,...,...,...,...,...,...
15979,38.8880,142.1520,48.3,4.2,2008-10-04,20:32:54,Japan,?funato
15980,43.5950,133.7750,431.3,4.6,2008-03-16,19:20:40,Russia,Lazo
15981,44.3420,141.2660,247.0,3.7,2008-02-21,13:05:10,Japan,Rumoi
15982,37.1030,135.0250,377.4,3.2,2008-05-02,02:07:07,Japan,Mikuni


In [0]:
df_J.loc[:,"Clasificación Profundidad"] = 0

for indice in range(len(df_J)):

    valor_prof =float(df_J["Profundidad"][indice])

    if valor_prof <= 70:
        df_J.loc[indice,"Clasificación Profundidad"] = "Superficial"

    elif valor_prof > 70 and valor_prof <= 299:
        df_J.loc[indice,"Clasificación Profundidad"] = "Media"
        
    elif valor_prof >= 300:
        df_J.loc[indice,"Clasificación Profundidad"] = "Profundo"

In [0]:
rangos = [(2, 2.9), (3, 3.9), (4, 4.9), (5, 5.9), (6, 6.9), (7, 7.9), (8, 9.9), (10, 15)]
etiquetas = ['Micro Magnitud', 'Menor Magnitud', 'Ligera Magnitud', 'Moderada Magnitud',
              'Fuerte Magnitud', 'Mayor Magnitud', 'Gran Magnitud', 'Magnitud Épica']

df_J['Clasificación Magnitud'] = pd.cut(df_J['Magnitud'], bins=[lim[0] for lim in rangos] + [rangos[-1][1]],
                               labels=etiquetas, right=False)

In [0]:
df_J = df_J[["Fecha" , "Hora", "Latitud", "Longitud", "País", "Locacion", "Profundidad", "Clasificación Profundidad", "Magnitud", "Clasificación Magnitud"]]
df_J = df_J.sort_values('Fecha')
df_J = df_J.reset_index(drop=True)

In [0]:
df_J['Fecha'] = pd.to_datetime(df_J['Fecha'])

In [0]:
Base_Final_Japon = df_J

In [0]:
Base_Final_Japon

,Fecha,Hora,Latitud,Longitud,País,Locacion,Profundidad,Clasificación Profundidad,Magnitud,Clasificación Magnitud
0,2008-01-01,05:32:40,44.0490,142.6850,Japan,Kamikawa,233.600,Media,3.7,Menor Magnitud
1,2008-01-01,16:20:06,42.4080,143.0570,Japan,Shizunai-furukawach?,55.900,Superficial,4.2,Ligera Magnitud
2,2008-01-01,11:12:08,39.0190,142.3410,Japan,Honshu,40.000,Superficial,4.2,Ligera Magnitud
3,2008-01-02,14:40:22,34.2070,135.0810,Japan,Minato,10.100,Superficial,3.4,Menor Magnitud
4,2008-01-02,10:13:49,33.3030,140.9170,Japan,Tateyama,47.000,Superficial,3.2,Menor Magnitud
...,...,...,...,...,...,...,...,...,...,...
15979,2023-12-03,20:25:34,39.5736,142.3764,Japan,Honshu,88.630,Media,4.2,Ligera Magnitud
15980,2023-12-04,08:45:55,37.0881,141.2152,Desconocido,None,60.211,Superficial,4.3,Ligera Magnitud
15981,2023-12-04,23:12:49,43.5428,145.7574,Japan,Nemuro,101.411,Media,4.3,Ligera Magnitud
15982,2023-12-05,12:45:05,33.4635,132.3792,Desconocido,None,55.487,Superficial,4.3,Ligera Magnitud


##Concatenar las Bases

In [0]:
Base_Final = pd.concat([Base_Final_Japon,Base_Final_Mexico,Base_Final_EEUU, Base_Final_Peru], axis = 0)

In [0]:
DB_Prueba = Base_Final

In [0]:
DB_Prueba

,Fecha,Hora,Latitud,Longitud,País,Locacion,Profundidad,Clasificación Profundidad,Magnitud,Clasificación Magnitud
0,2008-01-01,05:32:40,44.049,142.685,Japan,Kamikawa,233.6,Media,3.7,Menor Magnitud
1,2008-01-01,16:20:06,42.408,143.057,Japan,Shizunai-furukawach?,55.9,Superficial,4.2,Ligera Magnitud
2,2008-01-01,11:12:08,39.019,142.341,Japan,Honshu,40.0,Superficial,4.2,Ligera Magnitud
3,2008-01-02,14:40:22,34.207,135.081,Japan,Minato,10.1,Superficial,3.4,Menor Magnitud
4,2008-01-02,10:13:49,33.303,140.917,Japan,Tateyama,47.0,Superficial,3.2,Menor Magnitud
...,...,...,...,...,...,...,...,...,...,...
10884,2023-03-27,20:13:13,-10.53,-74.7,Perú,Junin,134.0,Media,4.6,Ligera Magnitud
10885,2022-10-28,23:19:58,-3.76,-76.53,Perú,Loreto,110.0,Media,4.8,Ligera Magnitud
10886,2023-05-22,00:05:45,-15.81,-73.32,Perú,Arequipa,92.0,Media,4.2,Ligera Magnitud
10887,2022-10-28,23:35:13,-14.61,-76.41,Perú,Ica,15.0,Superficial,5.2,Moderada Magnitud


##Descarga Base Final

In [0]:
import pandas as pd

# Supongamos que tienes un DataFrame de Pandas llamado df_pandas
df_spark = spark.createDataFrame(DB_Prueba)



In [0]:
df_spark.createOrReplaceTempView("Tabla_Temp")


In [0]:
from azure.storage.blob import BlobServiceClient
import tempfile
import os

# COMMAND ----------

# Configurar las credenciales de conexiÃ³n al Blob Storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=storagemainpf2;AccountKey=HaXeslJT0bFxZSyBjcN75dxyFVjFO7ncXM12BJncZZYSliO3d9yRuvFHCoPR3SlCF7O86v/QAnxF+ASt+vmNmg==;EndpointSuffix=core.windows.net"
container_name = "contenedorpf2"

# Crear el cliente de Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Obtener una referencia al contenedor
container_client = blob_service_client.get_container_client(container_name)

# Ruta y nombre del archivo en el Blob Storage
ruta_blob_storage = "/Tabla_Temp.csv"


# COMMAND ----------

# Guardar el DataFrame completo como un archivo CSV en el Blob Storage
DB_Prueba.to_csv(ruta_blob_storage, index=False)

# Cargar el archivo CSV completo al Blob Storage
with open(ruta_blob_storage, "rb") as file:
    container_client.upload_blob(name=ruta_blob_storage, data=file, overwrite=True)

## Conexión con Synapse

In [0]:
# Especifica los detalles de conexión a Azure Synapse Analytics
jdbc_hostname = "<nombre_servidor>.sql.azuresynapse.net"
jdbc_database = "<nombre_base_datos>"
jdbc_username = "<nombre_usuario>"
jdbc_password = "<contraseña>"
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:1433;database={jdbc_database};user={jdbc_username};password={jdbc_password}"

# Ejecuta una consulta SQL para seleccionar los datos de la tabla temporal
df_temp = spark.sql("SELECT * FROM nombre_tabla_temp")

# Escribe los resultados en Azure Synapse Analytics
df_temp.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "<nombre_tabla_destino>") \
    .option("user", jdbc_username) \
    .option("password", jdbc_password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .mode("append") \
    .save()